# Translate eisenscript to eisenxml using lark

In [92]:
from pathlib import Path
from lark import Lark
from lark.visitors import Interpreter, Transformer, v_args
parser = Lark.open('eisenscript.lark')

In [79]:
fn = Path("../tests/test_es/Examples/Ball.es")
fn.exists()

True

In [80]:
with open(fn, 'r') as f:
    es_txt = f.read()
tree = parser.parse(es_txt)
    

In [57]:
class EisenTreeToXml(Transformer):
   

    def trans(self, s):
        """
            x -> tx
            y -> ty
            z -> tz
            rx -> rx
            ry -> ry
            rz -> rz
            s n -> sa n
            s n n n -> s n n n

        IGNORING multiple values for scale for now
        """
        tid = s[0]
        tvalues = s[1]

        if tid in ("x", "y", "z"):
            tid = "t" + tid
        
        if tid in ('tx', 'ty', 'tz', 'rx', 'ry', 'rz', 'sa', 's'):
            tstr = tid + " " + tvalues + " "
            print(tstr)
        else:
            tstr = None
        return tstr
    
    def loop(self, s):
        """need to be producing xml at this stage"""
        print(s)
        return s


In [58]:
t = EisenTreeToXml().transform(tree)

[None, None]
ry 180 
['ry 180 ', None, None]
ty 0.4 
rx 1 
s 0.995 
['ty 0.4 ', 'rx 1 ', 's 0.995 ', None]
ty 0.4 
rx 1 
ry 1 
s 0.995 
['ty 0.4 ', 'rx 1 ', 'ry 1 ', 's 0.995 ', None]
ty 0.4 
rx 1 
rz -1 
s 1 
['ty 0.4 ', 'rx 1 ', 'rz -1 ', 's 1 ', None]
rx 15 
['rx 15 ']
ry 180 
['ry 180 ', None]


In [5]:
# try with Interpreter instead
class Es2xml(Interpreter):
    def trans(self, s):
        """
            x -> tx
            y -> ty
            z -> tz
            rx -> rx
            ry -> ry
            rz -> rz
            s n -> sa n
            s n n n -> s n n n

        IGNORING multiple values for scale for now
        """
        tid = s[0]
        tvalues = s[1]

        if tid in ("x", "y", "z"):
            tid = "t" + tid
        
        if tid in ('tx', 'ty', 'tz', 'rx', 'ry', 'rz', 'sa', 's'):
            tstr = tid + " " + tvalues + " "
            print(tstr)
        else:
            tstr = None
        return tstr

In [6]:
Es2xml().visit(tree)

TypeError: 'Tree' object is not subscriptable

In [8]:
# from https://github.com/lark-parser/lark/blob/master/tests/test_trees.py
from lark.tree import Tree
t = Tree('a', [Tree('b', []), Tree('c', []), 'd'])
print(t.pretty())

a
  b
  c
  d



In [9]:
class Interp1(Interpreter):
    def a(self, tree):
        return self.visit_children(tree) + ['e']

    def b(self, tree):
        return 'B'

    def c(self, tree):
        return 'C'

In [10]:
Interp1().visit(t)

['B', 'C', 'd', 'e']

So in this test 'a' has children 'b', 'c' and 'd'  
The method called for 'a' expressly calls  a visit on its children.  
The order of the list returned needs thinking about.  
'd' just gets added to the output because there is no explicit method for it, and its a terminal string


In [26]:
t = Tree('a', [Tree('b', [Tree('bb', [])]), Tree('c', []), 'd'])

In [27]:
print(t.pretty())

a
  b
    bb
  c
  d



In [34]:
class Interp2(Interpreter):
    def a(self, tree):
        return ['a root'] + self.visit_children(tree) + ['e root']

    def b(self, tree):
        return ['B child'] + self.visit_children(tree)

    def c(self, tree):
        return 'C child'

    def bb(self, tree):
        return 'BB grandchild'

In [35]:
Interp2().visit(t)

['a root', ['B child', 'BB grandchild'], 'C child', 'd', 'e root']

In [37]:
t

Tree('a', [Tree('b', [Tree('bb', [])]), Tree('c', []), 'd'])

In [36]:
tree

Tree(Token('RULE', 'main'), [Tree(Token('RULE', 'global_md'), [Token('NUMBER', '2000')]), Tree(Token('RULE', 'entry'), [Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'trans'), [Token('CID', 'a'), Token('VALUE', '0.9')]), Tree(Token('RULE', 'trans'), [Token('CID', 'hue'), Token('VALUE', '30')])]), Token('RULENAME', 'R1')])]), Tree(Token('RULE', 'ruledef'), [Token('RULENAME', 'R1'), Tree(Token('RULE', 'w_mod'), [Token('NUMBER', '10')]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'trans'), [Token('TID', 'x'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'rz'), Token('VALUE', '3')]), Tree(Token('RULE', 'trans'), [Token('TID', 'ry'), Token('VALUE', '5')])]), Token('RULENAME', 'R1')])]), Tree(Token('RULE', 'ruledef'), [Token('RULENAME', 'R1'), Tree(Token('RULE', 'w_mod'), [Token('NUMBER', '10')]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'trans'), [Token('TID', 'x'), Toke

In [20]:
print(tree.pretty())

main
  global_md	2000
  entry
    call
      loop
        trans
          a
          0.9
        trans
          hue
          30
      R1
  ruledef
    R1
    w_mod	10
    call
      loop
        trans
          x
          1
        trans
          rz
          3
        trans
          ry
          5
      R1
  ruledef
    R1
    w_mod	10
    call
      loop
        trans
          x
          1
        trans
          rz
          -3
        trans
          ry
          5
      R1
    call
      loop
        trans
          s
          1
          1
          0.1
      box



In eisen tree structure I have nodes for
 - main
 - global_md
 - entry
 - call
 - loop
 - trans 
 - w_mod
 - md_mod

 will need an Interpreter method for each which adds something to the xml tree  
 how do I get token values?
 look   @visit_children_decor

In [126]:
# minimal example
test = """ 2 * { x 1 y 2} seed
{ rx 1 ry 2} seed
rule seed  { 
  box 
}
"""

In [127]:
small = parser.parse(test)
small

Tree(Token('RULE', 'main'), [Tree(Token('RULE', 'entry'), [Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'count'), [Token('NUMBER', '2')]), Tree(Token('RULE', 'trans'), [Token('TID', 'x'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'y'), Token('VALUE', '2')])]), Token('RULENAME', 'seed')]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'trans'), [Token('TID', 'rx'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'ry'), Token('VALUE', '2')])]), Token('RULENAME', 'seed')])]), Tree(Token('RULE', 'ruledef'), [Token('RULENAME', 'seed'), Tree(Token('RULE', 'call'), [Token('RULENAME', 'box')])])])

In [128]:
print(small.pretty())

main
  entry
    call
      loop
        count	2
        trans
          x
          1
        trans
          y
          2
      seed
    call
      loop
        trans
          rx
          1
        trans
          ry
          2
      seed
  ruledef
    seed
    call	box



In [129]:

class Itest(Interpreter):
    
    def main(self, tree):
        print('main')
        
        return ['begin main'] + self.visit_children(tree) + ['end main']

    def entry(self, tree):
        print('entry')
        
        return ['begin entry'] + self.visit_children(tree) + ['end entry']

    def call(self, tree):
        print('call')
        print(tree)
       
        return ['begin call'] + self.visit_children(tree) + ['end call']
        
    @v_args(inline=True)
    def trans(self, tid, value):
        if tid in ("x", "y", "z"):
            tid = "t" + tid
        
        if tid in ('tx', 'ty', 'tz', 'rx', 'ry', 'rz', 'sa', 's'):
            tstr = tid + " " + value + " "
        else:
            tstr = None
        print('   trans ', tstr)

        return tstr
    
    def loop(self, tree):
        print('loop')
        
        return ' '.join(self.visit_children(tree))

    def ruledef(self, tree):
        print('ruledef')
        
        return ['begin ruledef'] + self.visit_children(tree) + ['end ruledef']

In [130]:
 it = Itest().visit(small)

main
entry
call
Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'count'), [Token('NUMBER', '2')]), Tree(Token('RULE', 'trans'), [Token('TID', 'x'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'y'), Token('VALUE', '2')])]), Token('RULENAME', 'seed')])
loop
   trans  tx 1 
   trans  ty 2 


TypeError: sequence item 0: expected str instance, list found

In [119]:
it

['begin main',
 ['begin entry',
  ['begin call', 'tx 1  ty 2 ', Token('RULENAME', 'seed'), 'end call'],
  ['begin call', 'rx 1  ry 2 ', Token('RULENAME', 'seed'), 'end call'],
  'end entry'],
 ['begin ruledef',
  Token('RULENAME', 'seed'),
  ['begin call', Token('RULENAME', 'box'), 'end call'],
  'end ruledef'],
 'end main']